### Watch thie video below to see a walkthrough of the Direct Lake Migration process
[![Direct Lake Migration Video](https://img.youtube.com/vi/gGIxMrTVyyI/0.jpg)](https://www.youtube.com/watch?v=gGIxMrTVyyI?t=495)


### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip install semantic-link-labs

### Import the library and set initial parameters

In [ ]:
import sempy_labs as labs
from sempy_labs import migration, directlake
import sempy_labs.report as rep

dataset_name = '' #Enter the import/DQ semantic model name
workspace_name = None #Enter the workspace of the import/DQ semantic model. It set to none it will use the current workspace.
new_dataset_name = '' #Enter the new Direct Lake semantic model name
new_dataset_workspace_name = None #Enter the workspace where the Direct Lake model will be created. If set to None it will use the current workspace.
lakehouse_name = None #Enter the lakehouse to be used for the Direct Lake model. If set to None it will use the lakehouse attached to the notebook.
lakehouse_workspace_name = None #Enter the lakehouse workspace. If set to None it will use the new_dataset_workspace_name.

### Create the [Power Query Template](https://learn.microsoft.com/power-query/power-query-template) file

This encapsulates all of the semantic model's Power Query logic into a single file.

In [ ]:
migration.create_pqt_file(dataset = dataset_name, workspace = workspace_name)

### Import the Power Query Template to Dataflows Gen2

- Open the [OneLake file explorer](https://www.microsoft.com/download/details.aspx?id=105222) and sync your files (right click -> Sync from OneLake)

- Navigate to your lakehouse. From this window, create a new Dataflows Gen2 and import the Power Query Template file from OneLake (OneLake -> Workspace -> Lakehouse -> Files...), and publish the Dataflows Gen2.

<div class="alert alert-block alert-info">
<b>Important!</b> Make sure to create the Dataflows Gen2 from within the lakehouse window. That will ensure that all the tables automatically map to that lakehouse as the destination. Otherwise, you will have to manually map each table to its destination individually.
</div>

### Create the Direct Lake model based on the import/DQ semantic model

Calculated columns are not migrated to the Direct Lake model as they are not supported in Direct Lake mode.

In [ ]:
import time
labs.create_blank_semantic_model(dataset = new_dataset_name, workspace = new_dataset_workspace_name, overwrite=False)

migration.migrate_calc_tables_to_lakehouse(
    dataset=dataset_name,
    new_dataset=new_dataset_name,
    workspace=workspace_name,
    new_dataset_workspace=new_dataset_workspace_name,
    lakehouse=lakehouse_name,
    lakehouse_workspace=lakehouse_workspace_name
)
migration.migrate_tables_columns_to_semantic_model(
    dataset=dataset_name,
    new_dataset=new_dataset_name,
    workspace=workspace_name,
    new_dataset_workspace=new_dataset_workspace_name,
    lakehouse=lakehouse_name,
    lakehouse_workspace=lakehouse_workspace_name
)
migration.migrate_calc_tables_to_semantic_model(
    dataset=dataset_name,
    new_dataset=new_dataset_name,
    workspace=workspace_name,
    new_dataset_workspace=new_dataset_workspace_name,
    lakehouse=lakehouse_name,
    lakehouse_workspace=lakehouse_workspace_name
)
migration.migrate_model_objects_to_semantic_model(
    dataset=dataset_name,
    new_dataset=new_dataset_name,
    workspace=workspace_name,
    new_dataset_workspace=new_dataset_workspace_name
)
migration.migrate_field_parameters(
    dataset=dataset_name,
    new_dataset=new_dataset_name,
    workspace=workspace_name,
    new_dataset_workspace=new_dataset_workspace_name
)
time.sleep(2)
labs.refresh_semantic_model(dataset=new_dataset_name, workspace=new_dataset_workspace_name)
migration.refresh_calc_tables(dataset=new_dataset_name, workspace=new_dataset_workspace_name)
labs.refresh_semantic_model(dataset=new_dataset_name, workspace=new_dataset_workspace_name)

### Show migrated/unmigrated objects

In [ ]:
migration.migration_validation(
    dataset=dataset_name,
    new_dataset=new_dataset_name, 
    workspace=workspace_name, 
    new_dataset_workspace=new_dataset_workspace_name
)

### Rebind all reports using the old semantic model to the new Direct Lake semantic model

In [ ]:
rep.report_rebind_all(
    dataset=dataset_name,
    dataset_workspace=workspace_name,
    new_dataset=new_dataset_name,
    new_dataset_workspace=new_dataset_workspace_name,
    report_workspace=None
)

### Rebind reports one-by-one (optional)

In [ ]:
report_name = '' # Enter report name which you want to rebind to the new Direct Lake model

rep.report_rebind(
    report=report_name,
    dataset=new_dataset_name,
    report_workspace=workspace_name,
    dataset_workspace=new_dataset_workspace_name)

### Show unsupported objects

In [ ]:
dfT, dfC, dfR = directlake.show_unsupported_direct_lake_objects(dataset = dataset_name, workspace = workspace_name)

print('Calculated Tables are not supported...')
display(dfT)
print("Learn more about Direct Lake limitations here: https://learn.microsoft.com/power-bi/enterprise/directlake-overview#known-issues-and-limitations")
print('Calculated columns are not supported. Columns of binary data type are not supported.')
display(dfC)
print('Columns used for relationship must be of the same data type.')
display(dfR)

### Schema check between semantic model tables/columns and lakehouse tables/columns

This will list any tables/columns which are in the new semantic model but do not exist in the lakehouse

In [ ]:
directlake.direct_lake_schema_compare(dataset=new_dataset_name, workspace=new_dataset_workspace_name)

### Show calculated tables which have been migrated to the Direct Lake semantic model as regular tables

In [ ]:
directlake.list_direct_lake_model_calc_tables(dataset=new_dataset_name, workspace=new_dataset_workspace_name)

### Repoint tables (partitions) to different source tables

In [ ]:
directlake.update_direct_lake_partition_entity(
    dataset=new_dataset_name,
    table_name=['Sales', 'Geography'], # Enter a list of table names to be repointed to the new source tables
    entity_name=['FactSales', 'DimGeography'], # Enter a list of the source tables (in the same order as table_name)
    schema=None,
    workspace=new_dataset_workspace_name,
)

### Repoint the model to a different lakehouse/warehouse etc.

In [ ]:
directlake.update_direct_lake_model_connection(
    dataset=new_dataset_name,
    workspace=new_dataset_workspace_name,
    source='MyLakehouse',
    source_type='Lakehouse', # 'Lakehouse' or 'Warehouse'
    source_workspace='MyLakehouseWorkspace',
    use_sql_endpoint=False
)

### Repoint tables within the model to a different lakehouse/warehouse etc.

In [ ]:
directlake.update_direct_lake_model_connection(
    dataset=new_dataset_name,
    workspace=new_dataset_workspace_name,
    source='MyLakehouse',
    source_type='Lakehouse',
    source_workspace='MyLakehouseWorkspace',
    use_sql_endpoint=False,
    tables=['Sales', 'Budget']  # Specify the tables to update to the new source
)